In [74]:
import math
import os
import sys
# from datetime import datetime
import pandas as pd
import numpy as np
import xarray as xr
import pickle
import netCDF4 as nc
import datetime

sys.path.append('D:\\mehdi\\WaterBalance_new\\')
from qdwb.evapotranspiration.et import *
from qdwb.evapotranspiration.asset import *
from qdwb.evapotranspiration.convert import *
from qdwb.primary_surface_flow.primary_surface_flow import *
from qdwb.primary_surface_flow.asset import *
from qdwb.primary_surface_flow.check import *
from qdwb.soil_content.soil_layers import *


In [75]:
data_path = "D:\\mehdi\\DailyData\\Data\\"

# list of name of files in folder
def list_of_files(data_path):    
    files = os.listdir(data_path)
    return files

all_list_files = list_of_files(data_path)

# PET

In [76]:
para = ["P", "Tmin", "Tmax"]

list_files = {}

for p in para:
    
    tmp = pd.DataFrame(
        {
            "file": [x for x in all_list_files if x.startswith(p)],
            "year": [int(x.split("_")[1][0:4]) for x in all_list_files if x.startswith(p)]
        }
        
    )
    
    tmp.sort_values(by = "year", inplace = True)
    
    list_files[p] = tmp



In [77]:
# with open("D:\mehdi\DailyData\land_use.pkl", "rb") as f:
#     land_use = pickle.load(f)

# land_use

# land_use["Plant Date"] = np.datetime64("2022") + land_use['Plant Date'].astype('timedelta64[M]') + np.timedelta64(0, 'D')

In [78]:
# df = land_use.to_dataframe()
# df = df.reset_index()
# column_titles = df.columns.tolist()
# lll = df['cultivated'].to_list()

# unique_list = pd.Series(lll).unique()
# unique_list


In [79]:

# math.isnan(np.unique(land_use['Plant Date'].sel(time = "2022-01-01")[:,:])[-1])

In [80]:
# {year}-{int(x):02d}-{1:02d}

def generate_date(x, year):
    
    if math.isnan(x):
        return "nan"
    else:
        return f"{year}-{int(x):02d}-01"



# plant_date_day = np.vectorize(generate_date)(
#     x = land_use['Plant Date'].sel(time = "2022-01-01")[:,:],
#     year = 2022
# )

# plant_date_day = np.where(plant_date_day == "nan", np.nan, plant_date_day)



In [138]:
selected_years = [2010, 2011]

with open("D:\mehdi\DailyData\land_use.pkl", "rb") as f:
    land_use = pickle.load(f)
land_use = land_use.rename({'x': 'lon','y': 'lat'})

land_use["Total"].data = np.where(land_use["Total"].data == 0.0, np.nan, land_use["Total"].data)

wp = xr.open_dataset("D:\mehdi\DailyData\wilting_point_1979001.nc")
wp = wp.rename({'x': 'lon','y': 'lat'})

fc = xr.open_dataset("D:\mehdi\DailyData\coefficient_field_capacity_1979001.nc")
fc = fc.rename({'x': 'lon','y': 'lat'})

cn = xr.open_dataset("D:\mehdi\DailyData\curve_number_1979001.nc")

cn = cn.fillna(0)
cn = cn.rename({'x': 'lon','y': 'lat'})

for year in selected_years:
    
    Pre = xr.open_dataset(f"{data_path}{list_files['P'][list_files['P']['year'] == year]['file'].values[0]}")    
    Tmin = xr.open_dataset(f"{data_path}{list_files['Tmin'][list_files['Tmin']['year'] == year]['file'].values[0]}")    
    Tmax = xr.open_dataset(f"{data_path}{list_files['Tmax'][list_files['Tmax']['year'] == year]['file'].values[0]}")    
    lat = np.repeat(Tmin.lat, Tmin.lon.size).values.reshape(Tmin.lat.size, Tmin.lon.size)    
    lon = np.repeat(Tmin.lon, Tmin.lat.size).values.reshape(Tmin.lat.size, Tmin.lon.size)
    
    tmpDataArray = Tmin.copy(deep = True)
    tmpDataArray = tmpDataArray.rename({'air_temperature': 'temp'})
    tmpDataArray["temp"].values = np.empty(shape=tmpDataArray["temp"].shape)
        
    Ra = tmpDataArray.copy(deep = True)    
    Ra = Ra.rename({'temp': 'extraterrestrial_radiation'})
    
    ET = tmpDataArray.copy(deep = True)    
    ET = ET.rename({'temp': 'et_hargreaves_samani'})
    
    TE = tmpDataArray.copy(deep = True)    
    TE = TE.rename({'temp': 'total_evaporable_water'})
    
    MRF = tmpDataArray.copy(deep = True)    
    MRF = MRF.rename({'temp': 'moisture_reduction_function'})
    
    f_adjusted = tmpDataArray.copy(deep = True)    
    f_adjusted = f_adjusted.rename({'temp': 'moisture_reduction_function'})
    
    ET_actual = tmpDataArray.copy(deep = True)    
    ET_actual = ET_actual.rename({'temp': 'actual_evapotranspiration'})
    
    K_E = tmpDataArray.copy(deep = True)    
    K_E = K_E.rename({'temp': 'available_evaporable_water'})
    
    E_actual = tmpDataArray.copy(deep = True)    
    E_actual = E_actual.rename({'temp': 'actual_evaporation'})
    
    ET_total_actual = tmpDataArray.copy(deep = True)    
    ET_total_actual = ET_total_actual.rename({'temp': 'total_actual_evapotranspiration'})
    
    RUN_OFF = tmpDataArray.copy(deep = True)    
    RUN_OFF = RUN_OFF.rename({'temp': 'runoff'})
    
    INFILTRATION = tmpDataArray.copy(deep = True)    
    INFILTRATION = INFILTRATION.rename({'temp': 'infiltration'})
    
    CROP_COEFFICIENT = tmpDataArray.copy(deep = True)    
    CROP_COEFFICIENT = CROP_COEFFICIENT.rename({'temp': 'crop_coefficient'})
    
    SWC_EVAPORATION = tmpDataArray.copy(deep = True)    
    SWC_EVAPORATION = SWC_EVAPORATION.rename({'temp': 'current_swc_evaporation_layer'})
    
    SWC_TRANSPIRATION = tmpDataArray.copy(deep = True)    
    SWC_TRANSPIRATION = SWC_TRANSPIRATION.rename({'temp': 'current_swc_transpiration_layer'})
    
    SWC_TRANSITION = tmpDataArray.copy(deep = True)    
    SWC_TRANSITION = SWC_TRANSITION.rename({'temp': 'current_swc_transition_layer'})
    
    EVAPORATION_TO_TRANSPIRATION = tmpDataArray.copy(deep = True)    
    EVAPORATION_TO_TRANSPIRATION = EVAPORATION_TO_TRANSPIRATION.rename({'temp': 'current_evaporation_layer_to_transpiration_layer'})
    
    EVAPORATION_TO_TRANSITION = tmpDataArray.copy(deep = True)    
    EVAPORATION_TO_TRANSITION = EVAPORATION_TO_TRANSITION.rename({'temp': 'current_evaporation_layer_to_transition_layer'})
    
    TRANSPIRATION_TO_TRANSITION = tmpDataArray.copy(deep = True)    
    TRANSPIRATION_TO_TRANSITION = TRANSPIRATION_TO_TRANSITION.rename({'temp': 'current_transpiration_layer_to_transition_layer'})
    
    TRANSPIRATION_ADJUSTED = tmpDataArray.copy(deep = True)    
    TRANSPIRATION_ADJUSTED = TRANSPIRATION_ADJUSTED.rename({'temp': 'transpiration'})
    
    EVAPORATION_ADJUSTED = tmpDataArray.copy(deep = True)    
    EVAPORATION_ADJUSTED = EVAPORATION_ADJUSTED.rename({'temp': 'evaporation'})
    
    IRRIGATION_REQUIREMENT = tmpDataArray.copy(deep = True)    
    IRRIGATION_REQUIREMENT = IRRIGATION_REQUIREMENT.rename({'temp': 'irrigation_requirement'})
    
    DEEP_PERCOLATION = tmpDataArray.copy(deep = True)    
    DEEP_PERCOLATION = DEEP_PERCOLATION.rename({'temp': 'deepPercolation'})
    
    for j_day in range(1, Tmin.time.size + 1):
        
        print(year, j_day)
        
        if year == selected_years[0] and j_day == 1:
            soil_water_content = xr.open_dataset("D:\mehdi\DailyData\soil_water_content_1979001.nc")        
            soil_water_content = soil_water_content.fillna(0)         
            soil_water_content = soil_water_content.rename({'x': 'lon','y': 'lat'})
            
            soil_water_content_evaporation = soil_water_content["volumetric_soil_water_layer_1"].sel(time = "1979-01-01")[:, :]
            soil_water_content_transpiration = soil_water_content["volumetric_soil_water_layer_2"] + soil_water_content["volumetric_soil_water_layer_3"]
            soil_water_content_transpiration = soil_water_content_transpiration.sel(time = "1979-01-01")[:, :]
            soil_water_content_transition = soil_water_content["volumetric_soil_water_layer_4"].sel(time = "1979-01-01")[:, :]
            
            
            
            constant_value_for_AE = 6
            # AE = xr.Dataset(
            #     data_vars={
            #         "available_evaporable_water": (("time", "lat", "lon"), np.full((len(soil_water_content.time), len(soil_water_content.lat), len(soil_water_content.lon)), constant_value_for_AE))
            #     },
            #     coords={
            #         "lat": soil_water_content.lat,
            #         "lon": soil_water_content.lon,
            #         "time": soil_water_content.time
            #     },
            # )
            # AE = AE.to_array()
            
            AE = np.zeros((len(soil_water_content.lat), len(soil_water_content.lon))) + constant_value_for_AE
                    
        else:
            
            soil_water_content_evaporation = soil_water_content_day[0]
            soil_water_content_evaporation = np.vectorize(lambda d: 0 if np.isnan(d) else d)(
                d = soil_water_content_evaporation
            )
            soil_water_content_transpiration = soil_water_content_day[1]
            soil_water_content_transition = soil_water_content_day[2]
            
            # TODO: AE???
            AE = np.vectorize(available_evaporable_water)(
                e_a = e_actual_day,
                infiltration = run_off_and_infiltration_day[1],
                available_evaporable_water_in_previous_step = available_evaporable_water_day
            )
            

        
        date_object = datetime.datetime.strptime(f"{year}{j_day}", '%Y%j').date()
        model_date = date_object.strftime("%Y-%m-%d")
        
        pre_day = Pre.to_array()[0, j_day - 1, :, :]
        
        Tmin_day = Tmin.to_array()[0, j_day - 1, :, :]
        
        Tmax_day = Tmax.to_array()[0, j_day - 1, :, :]
        
        Tmean_day = (Tmin_day + Tmax_day) / 2
        
        ra_day = np.vectorize(extraterrestrial_radiation)(
            inverse_relative_distance_earth_sun(
                julian_date = j_day
            ),
            sunset_hour_angle(
                latitude=convert_degrees2radians(lat),
                solar_declination = solar_declination(
                    julian_date = j_day
                )
            ),
            convert_degrees2radians(
                degrees = lat
            ),
            solar_declination(
                julian_date = j_day
            )            
        )
        
        Ra["extraterrestrial_radiation"][j_day - 1, :, :] = ra_day
                
        et_day_hargreaves_samani = np.vectorize(ReferenceEvapotranspiration.hargreaves_samani)(
            tmin = Tmin_day,
            tmax = Tmax_day,
            tmean = Tmean_day,
            ra = ra_day
        )
        et_day_hargreaves_samani = np.vectorize(lambda d: 0 if d < 0 else d)(
            d = et_day_hargreaves_samani
        )
        
        ET["et_hargreaves_samani"][j_day - 1, :, :] = et_day_hargreaves_samani
        
        total_evaporable_water_day = np.vectorize(available_water)(
            permanent_wilting_point_wet = wp["b0"].sel(time = "1979-01-01")[:, :],
            field_capacity_wet = fc["b0"].sel(time = "1979-01-01")[:, :],
            soil_depth = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000
        )
        
        TE["total_evaporable_water"][j_day - 1, :, :] = total_evaporable_water_day
        
        
        
        f_day = np.vectorize(moisture_reduction_function)(
            soil_wetness_in_previous_step = soil_water_content_transpiration,
            permanent_wilting_point_wet = wp["b0"].sel(time = "1979-01-01")[:, :],
            field_capacity_wet = fc["b0"].sel(time = "1979-01-01")[:, :],
            soil_depth = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000
        )
        MRF["moisture_reduction_function"][j_day - 1, :, :] = f_day
        
        f_adjusted_day = np.vectorize(lambda d: 0 if d < 0 else d)(
            d = f_day
        )
        
        f_adjusted["moisture_reduction_function"][j_day - 1, :, :] = f_adjusted_day
        
        plant_date_day = np.vectorize(generate_date)(
            x = land_use['Plant Date'].sel(time = "2022-01-01")[:,:],
            year = year
        )
        plant_date_day = np.where(plant_date_day == "nan", pd.NA, plant_date_day)        
        # plant_date_day = np.where(pd.isnull(plant_date_day), np.nan, plant_date_day)
        plant_date_day = np.where(pd.isnull(plant_date_day), "1960-01-01", plant_date_day)
        
        crop_coefficient_day = np.vectorize(PotentialEvapotranspiration.calculate_single_crop_coefficient_for_linear_changes_steps)(
            crop_coefficient_ini = land_use["Kc init"].sel(time = "2022-01-01")[:,:],
            crop_coefficient_mid = land_use["Kc mid"].sel(time = "2022-01-01")[:,:],
            crop_coefficient_end = land_use["Kc end"].sel(time = "2022-01-01")[:,:],
            length_ini_crop = land_use["Init. (Lini)"].sel(time = "2022-01-01")[:,:],
            length_dev_crop = land_use["Dev. (Ldev)"].sel(time = "2022-01-01")[:,:],
            length_mid_crop = land_use["Mid (Lmid)"].sel(time = "2022-01-01")[:,:],
            length_late_crop = land_use["Late (Llate)"].sel(time = "2022-01-01")[:,:],
            plant_date = plant_date_day[:,:],
            modeling_date = model_date
        )
        
        
        CROP_COEFFICIENT["crop_coefficient"][j_day - 1, :, :] = crop_coefficient_day
        
        
        et_actual_day = np.vectorize(ActualEvapotranspiration.et_covered)(
            moisture_reduction_function = f_adjusted_day,
            crop_coefficient = CROP_COEFFICIENT["crop_coefficient"].sel(time = model_date)[:,:],
            crop_cover = 0.4,
            reference_crop_evapotranspiration = et_day_hargreaves_samani
        )
        ET_actual["actual_evapotranspiration"][j_day - 1, :, :] = et_actual_day
        
        
        available_evaporable_water_day = AE
        # available_evaporable_water_day = AE[:, :]
        
        k_e_day = np.vectorize(ratio_of_actual_evaporable_water_to_total_evaporable_water)(
            available_water = total_evaporable_water_day,
            available_evaporable_water = available_evaporable_water_day
        )
        
        K_E["available_evaporable_water"][j_day - 1, :, :] = k_e_day
        
        
        e_actual_day = np.vectorize(ActualEvapotranspiration.e_noncovered)(
            ratio_of_actual_evaporable_water_to_total_evaporable_water = k_e_day,
            crop_cover = 0.4,
            reference_crop_evapotranspiration = et_day_hargreaves_samani
        )
        
        E_actual["actual_evaporation"][j_day - 1, :, :] = e_actual_day
        
        et_total_actual_day = np.vectorize(ActualEvapotranspiration.et_QDWB)(
            evaporation_noncovered_areas = e_actual_day,
            evapotranspiration_covered_areas = et_actual_day
        )
        
        ET_total_actual["total_actual_evapotranspiration"][j_day - 1, :, :] = et_total_actual_day
        
        model_date_array = np.array([datetime.datetime.strptime(model_date, '%Y-%m-%d')] * plant_date_day.shape[0] * plant_date_day.shape[1]).reshape(plant_date_day.shape[0], plant_date_day.shape[1])
        plant_date_day_array = np.vectorize(lambda d: datetime.datetime.strptime(d, '%Y-%m-%d'))(
            d = plant_date_day
        )
        total_adjusted = np.where(np.isnan(land_use["Total"][0].data), 0 , land_use["Total"][0].data)
        
        end_date_of_growing = np.vectorize(lambda x , y: x + datetime.timedelta(days=y))(
            x = plant_date_day_array,
            y = total_adjusted
        )
        growing_season = np.vectorize(lambda m, a, b: True if m >= a and m <= b else False)(
            m = model_date_array,
            a = plant_date_day_array,
            b = end_date_of_growing
        )
        
        # ### run off
        
        run_off_and_infiltration_day = np.vectorize(PrimarySurfaceFlow.scs)(
            precipitation = pre_day,
            curve_number = cn["dry"].sel(time = "1979-01-01")[:, :],
            rsa = True,
            antecedent_precipitation = soil_water_content_evaporation,
            is_growing_season = growing_season
        )
        RUN_OFF["runoff"][j_day - 1, :, :] = run_off_and_infiltration_day[0]
        
        INFILTRATION["infiltration"][j_day - 1, :, :] = run_off_and_infiltration_day[1]
        
        # ### soil water content

        
        # soil_water_content_day = np.vectorize(waterSoilContent)(
        #     covered = land_use["cultivated"].sel(time = "2022-01-01")[:,:],
        #     infiltration= run_off_and_infiltration_day[1],
        #     evaporation= e_actual_day,
        #     init_swc_evaporation_layer = soil_water_content_evaporation,
        #     init_swc_transition_layer = soil_water_content_transition,
        #     z_evaporation_layer = 100,
        #     z_transition_layer = 1000,
        #     fc_evaporation_layer = fc["b10"].sel(time = "1979-01-01")[:, :],
        #     fc_transition_layer = fc["b200"].sel(time = "1979-01-01")[:, :],
        #     pwp_evaporation_layer = wp["b10"].sel(time = "1979-01-01")[:, :],
        #     pwp_transition_layer = wp["b200"].sel(time = "1979-01-01")[:, :],
        #     stress_coefficient = 0.7,
        #     MAD = 0.3,
        #     transpiration = et_actual_day,
        #     init_swc_transpiration_layer = soil_water_content_transpiration,
        #     pwp_transpiration_layer = wp["b100"].sel(time = "1979-01-01")[:, :],
        #     z_transpiration_layer = land_use["Maximum Crop Height (h) (m)"].sel(time = "2022-01-01")[:, :]*1000,
        #     fc_transpiration_layer = fc["b100"].sel(time = "1979-01-01")[:, :]
        # )
        
        # SWC_EVAPORATION["current_swc_evaporation_layer"][j_day - 1, :, :] = soil_water_content_day[0]
        # SWC_TRANSPIRATION["current_swc_transpiration_layer"][j_day - 1, :, :] = soil_water_content_day[1]
        # SWC_TRANSITION["current_swc_transition_layer"][j_day - 1, :, :] = soil_water_content_day[2]
        # EVAPORATION_TO_TRANSPIRATION["current_evaporation_layer_to_transpiration_layer"][j_day - 1, :, :] = soil_water_content_day[3]
        # EVAPORATION_TO_TRANSITION["current_evaporation_layer_to_transition_layer"][j_day - 1, :, :] = soil_water_content_day[4]
        # TRANSPIRATION_TO_TRANSITION["current_transpiration_layer_to_transition_layer"][j_day - 1, :, :] = soil_water_content_day[5]
        # TRANSPIRATION_ADJUSTED["transpiration"][j_day - 1, :, :] = soil_water_content_day[6]
        # EVAPORATION_ADJUSTED["evaporation"][j_day - 1, :, :] = soil_water_content_day[7]
        # IRRIGATION_REQUIREMENT["irrigation_requirement"][j_day - 1, :, :] = soil_water_content_day[8]
        # DEEP_PERCOLATION["deepPercolation"][j_day - 1, :, :] = soil_water_content_day[9]
        
        
    
        
        
        break
    # Ra.to_netcdf(f"{data_path}Ra_{year}.nc")
    
    # ET.to_netcdf(f"{data_path}ET_{year}.nc")
        
    
    break


2010 1


c:\Users\psh2610\AppData\Local\miniconda3\lib\site-packages\numpy\lib\function_base.py:2455: RuntimeWarning: invalid value encountered in func (vectorized)
  outputs = ufunc(*inputs)


In [141]:
ru = run_off_and_infiltration_day[0]
ru[0]

array([ 5.4375    ,  7.        ,  7.1875    ,  7.5625    ,  9.8125    ,
        9.8125    , 16.0625    , 17.0625    , 17.0625    ,  0.06081779,
        9.71607328,  3.366245  ,  2.57592978,  2.76799235,  7.54959442,
        9.51529269, 13.64420664, 14.42132756, 11.59122153,  5.89814078,
        7.16088929, 13.22820828, 11.00019091, 11.67195332, 10.95566774,
       11.83516382, 16.32949547, 14.88749387, 23.77356133, 19.59895614,
       25.17024343, 20.25621134, 33.2850944 , 28.58623137, 24.17555737,
       17.24099227, 18.90221521, 19.981895  , 20.28237197, 19.01905045,
       19.7706599 , 19.40032487, 19.43381443, 19.43381443, 19.43381443,
       18.74888325, 19.20012962, 19.00120359, 19.84922975, 24.22430942,
       24.1912939 , 22.71265701, 22.74521536, 23.76459177, 19.26918098,
       18.5203413 , 16.98806397, 17.83024289, 19.51614502, 20.02125702,
       19.92871254, 18.8382941 , 19.18432179, 19.98200998, 18.65724276,
       17.63673005, 18.26479892, 29.33710952, 30.02726821, 22.86

In [132]:
logic_grow = np.vectorize(lambda m, a, b: True if m >= a and m <= b else False)(
    m = model_date_array,
    a = plant_date_day_array,
    b = end_date_of_growing
)
# np.where(model_date_array > plant_date_day_array and model_date_array < end_date_of_growing , True, False)

array([False,  True])

In [105]:
land_use["Total"][0]

<xarray.DataArray 'Total' (lat: 200, lon: 600)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lon      (lon) float64 -124.9 -124.8 -124.8 -124.6 ... -65.25 -65.15 -65.05
  * lat      (lat) float64 44.95 44.85 44.75 44.65 ... 25.35 25.25 25.15 25.05
    time     datetime64[ns] 2022-01-01

In [124]:
total_p = np.where(np.isnan(land_use["Total"][0].data), 0 , land_use["Total"][0].data)
dffff = np.vectorize(lambda d , b: d + datetime.timedelta(days=b))(
    d = plant_date_day_array,
    b = total_p
)
dffff[0]

array([datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1, 1, 0, 0),
       datetime.datetime(1960, 1

In [92]:
model_date_array

array([[datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0), ...,
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0)],
       [datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0), ...,
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0)],
       [datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0), ...,
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0)],
       ...,
       [datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0),
        datetime.datetime(2010, 1, 1, 0, 0), ...,
        datetime.dat

In [87]:
crop_coefficient_day[0]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan, 0.4 ,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan, 0.4 ,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan, 0.4 , 0.4 , 0.4 , 0.4 , 0.4 , 0.4 ,  nan,
        nan,  nan,  nan,  nan, 0.4 , 0.4 ,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan, 0.4 , 0.4 , 0.4 , 0.4 ,  nan,  nan,
        nan, 0.4 , 0.4 , 0.4 , 0.4 , 0.4 ,  nan,  nan,  nan,  nan,  nan,
        nan,  nan, 0.4 , 0.4 , 0.4 , 0.4 ,  nan,  nan,  nan, 0.4 , 0.4 ,
       0.4 , 0.4 , 0.4 , 0.4 , 0.4 , 0.4 , 0.4 , 0.4 ,  nan,  nan,  nan,
        nan,  nan, 0.4 ,  nan,  nan,  nan,  nan,  nan, 0.4 , 0.4 , 0.4 ,
       0.4 , 0.4 ,  nan, 0.4 , 0.4 ,  nan,  nan,  n

In [11]:
pre_day

<xarray.DataArray (lat: 200, lon: 600)>
array([[5.4375, 7.    , 7.1875, ..., 2.5625, 2.4375, 2.4375],
       [5.8125, 7.    , 7.    , ..., 4.9375, 5.    , 4.9375],
       [5.8125, 7.    , 7.    , ..., 5.125 , 5.125 , 5.0625],
       ...,
       [0.0625, 0.0625, 0.0625, ..., 0.625 , 0.625 , 0.625 ],
       [0.0625, 0.0625, 0.0625, ..., 0.625 , 0.625 , 0.625 ],
       [0.125 , 0.0625, 0.0625, ..., 0.5625, 0.5625, 0.6875]],
      dtype=float32)
Coordinates:
  * lon       (lon) float32 -124.9 -124.8 -124.8 -124.6 ... -65.25 -65.15 -65.05
  * lat       (lat) float32 44.95 44.85 44.75 44.65 ... 25.35 25.25 25.15 25.05
    time      datetime64[ns] 2010-01-01
    variable  <U13 'precipitation'
Attributes:
    history:  Created on 2021-01-12 22:09

In [12]:
cn["dry"].sel(time = "1979-01-01")[:, :]

<xarray.DataArray 'dry' (lat: 200, lon: 600)>
array([[ 0.,  0.,  0., ..., 54., 55., 57.],
       [ 0.,  0.,  0., ..., 56., 57., 54.],
       [ 0.,  0.,  0., ..., 56., 55., 55.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])
Coordinates:
  * lon      (lon) float32 -124.9 -124.8 -124.8 -124.6 ... -65.25 -65.15 -65.05
  * lat      (lat) float32 44.95 44.85 44.75 44.65 ... 25.35 25.25 25.15 25.05
    time     datetime64[ns] 1979-01-01
Attributes:
    AREA_OR_POINT:           Area
    TIFFTAG_RESOLUTIONUNIT:  1 (unitless)
    TIFFTAG_XRESOLUTION:     1
    TIFFTAG_YRESOLUTION:     1
    grid_mapping:            spatial_ref

In [13]:
soil_water_content_evaporation


<xarray.DataArray 'volumetric_soil_water_layer_1' (lat: 200, lon: 600)>
array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., 23.66257985,
        18.92695745, 24.98704274],
       [ 0.        ,  0.        ,  0.        , ..., 22.92322159,
        26.0312144 , 25.98132451],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])
Coordinates:
  * lon          (lon) float32 -124.9 -124.8 -124.8 ... -65.25 -65.15 -65.05
  * lat          (lat) float32 44.95 44.85 44.75 44.65 ... 25.25 25.15 25.05
    time         datetime64[ns] 1979-01-01
    spatial_ref  int32 ...

In [72]:
plant_date_day = np.where(pd.isnull(plant_date_day), "1960-01-01", plant_date_day)
plant_date_day[0]

array(['1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '2010-04-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '2010-04-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-01-01',
       '1960-01-01', '1960-01-01', '1960-01-01', '1960-

In [66]:
plant_date_day

array([[<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       ...,
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>]], dtype=object)

In [60]:
import datetime

model_date_array = np.array([datetime.datetime.strptime(model_date, '%Y-%m-%d')] * plant_date_day.shape[0] * plant_date_day.shape[1]).reshape(plant_date_day.shape[0], plant_date_day.shape[1])



array([[datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0), ...,
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0)],
       [datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0), ...,
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0)],
       [datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0), ...,
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0)],
       ...,
       [datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0),
        datetime.datetime(2010, 1, 11, 0, 0), ...,

In [67]:
type(plant_date_day[0][0])

pandas._libs.missing.NAType

In [68]:
np.vectorize(datetime.datetime.strptime)(
    plant_date_day,
    '%Y-%m-%d'
)

TypeError: strptime() argument 1 must be str, not NAType

In [48]:
# Number of Day
land_use["Total"].data[0]

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [80]:
def check_date(plant_date, n_day, model_date):
    for date_ii in plant_date:
        if pd.isna(date_ii):
            return False
        model_date_day = date(int(model_date[:4]), int(model_date[5:7]), int(model_date[8:]))
        pla_date = date(int(date_ii[:4]), int(date_ii[5:7]), int(date_ii[8:]))
        if model_date_day > pla_date and model_date_day < pla_date + timedelta(days=n_day):
            return True
    return False

In [83]:
plll = '2010-04-01'

iiiii = np.vectorize(check_date)(
    plant_date = plant_date_day,
    n_day = land_use["Total"][:,:],
    model_date = '2010-04-01'
)

TypeError: 'NAType' object is not iterable

In [81]:
plant_date_day

array([[<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       ...,
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>],
       [<NA>, <NA>, <NA>, ..., <NA>, <NA>, <NA>]], dtype=object)

In [72]:
date(int(model_date[:4]), int(model_date[5:7]), int(model_date[8:]))


# print(pd.isna(pd.NA))


igs = np.vectorize(lambda d: False if pd.isna(d) else date(int(d[:4]), int(d[5:7]), int(d[8:])))(
    d = plant_date_day
)
igs[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
        True,  True, False, False, False, False, False, False, False,
       False, False, False,  True,  True,  True,  True, False, False,
       False,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False,  True,  True,  True,  True, False,
       False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [65]:
plll = '2010-04-01'
plll_2 = '2010-03-02'
pld = date(int(plll[:4]), int(plll[5:7]), int(plll[8:]))
pld_2 = date(int(plll_2[:4]), int(plll_2[5:7]), int(plll_2[8:]))

print(pld_2 > pld)

False
